# Arrays
Multi-dimensional (heap-allocated) arrays are standard library data structures in Julia, with a rich set of tools for manipulation, reshaping, slices, views.

Manual: [Arrays](https://docs.julialang.org/en/v1/manual/arrays/)

In [ ]:
a = [1 2 3 1; 4 5 6 4; 7 8 9 7]
print("a: ")
display(a)
;

## Array Indexing
Arrays in Julia:
- stored __*column-major*__
- indexed starting at 1.

In [ ]:
@show a[1,1]
@show a[3,2]
;

In [ ]:
print("a_flat:")
a_flat = reshape(a, length(a))
display(a_flat)
;

In [ ]:
a_flat[1] = -1
display(a_flat)
display(a)
;

In [ ]:
# NOTE: This is just to demonstrate the array ordering;
# There are very few contexts in Julia in which raw pointers are needed
println("@a[1,2]:    ", pointer(view(a, 1, 2)))
println("@a_flat[4]: ", pointer(a_flat, 4))
@show unsafe_load(pointer(a_flat, 4))
;

In [ ]:
a_T = transpose(a)
display(a_T)
println("@a_T[2,1]:  ", pointer(view(a_T, 2, 1)))
;

So, when nesting loops, remember that the first index should be the fastest.

In [ ]:
for j in 1:3
    for i in 1:3
        println(i, ", ", j, ": ", a[i,j])
    end
end

__*WARNING*__: reshape and transpose are adapters but *slicing makes a copy!*

Use views to avoid copying.

In [ ]:
a_slice = a[1, :]
a_slice[2] = 100
@show typeof(a_slice)
display(a)

a_view = @view a[1, :]
a_view[2] = 100
@show typeof(a_view)
display(a)

## Array Broadcasting
Unary and binary functions on individual elements can be broadcast to all elements of an array (or two arrays). Nested broadcast operations are fused into one. The `@.` macro can be used to broadcast all operations in an expression without using the `.` everywhere.

In [ ]:
function bc()
    N = 20_000_000
    u = randn(N)
    v = randn(N)
    w = randn(N)

    println()

    # free fusion
    @time begin
        tmp = u + v
        fin = tmp + w
    end
    @time u + v + w

    println()

    @time begin
        # typical non-optimized result
        tmp1 = sin.(u)
        tmp2 = cos.(v)
        tmp3 = tmp1 .* tmp2
        tmp4 = cos.(w)
        tmp5 = sin.(tmp4)
        fin1 = tmp3 .* tmp5
    end
    @time begin
        fin2 = sin.(u) .* cos.(v) .* sin.(cos.(w))
    end
    @time begin
        # This is basically what the above expression is expanded to
        fin3 = broadcast((x,y,z) -> sin(x) * cos(y) * sin(cos(z)), u, v, w)
    end
    @time begin
        # Pre-allocate in order to use .=
        fin4 = similar(u)
        # Equivalent to
        # fin4 .= sin.(u) .* cos.(v) .* sin.(cos.(w))
        @. fin4 = sin(u) * cos(v) * sin(cos(w))
    end

    @show (fin1 == fin2 == fin3 == fin4)
end
bc()
bc()
bc()
;

This is handy for non-numerical applications as well...

In [ ]:
names = ["Brigid", "Thecla", "Hildegard"]
names .= "Saint " .* names .* ", pray for us."
display(names)
;